## RDKit in Snowpark

The following notebook covers examples of using RDKit with Snowpark. 

In [ ]:
#Standard Imports
import json
import pandas as pd
import numpy as np
import cachetools

#Snowflake imports: 
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
import snowflake.snowpark.types as T

#Rdkit specific libraries
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

In [ ]:
#Environment Set-Up: Fill in your creds.json file with your corresponding account credentials:  
with open('./creds.json') as f:
    data = json.load(f)
    USERNAME = data['user']
    PASSWORD = data['password']
    SF_ACCOUNT = data['account']
    
connection_parameters = {
    "account": SF_ACCOUNT,
    "user": USERNAME,
    "password": PASSWORD,
    "role": "ACCOUNTADMIN"
}

session = Session.builder.configs(connection_parameters).create()

Set up Database context: 

In [ ]:
session.sql('CREATE OR REPLACE DATABASE RDKIT').collect()
session.sql('CREATE OR REPLACE WAREHOUSE RDKIT_WH').collect()
session.sql('CREATE OR REPLACE STAGE TMP').collect()

### Loading Data: 

To run some sample computations, we will be loading our RDKIT databases with some data containing SMILES strings. Note that at the moment, Snowflake does not support arbitrary python objects as a type for a particular column.

The first dataset we will load into our database is provided by ChEMBL. Click on the download csv link on the following link: https://www.ebi.ac.uk/chembl/g/#search_results/all. 

In [ ]:
df = pd.read_csv('../smiles.csv', delimiter=';')
df.head()

In [ ]:
df.columns

In [ ]:
#Select only those columns that are required: 
cols = ['ChEMBL ID', 'Molecular Formula', 'Smiles']
df = df[cols]
df.head()

In [ ]:
df.shape

In [ ]:
#Drop rows with null entries
df.dropna(inplace = True)
df.shape

In [ ]:
#upload data to snowflake: 
session.write_pandas(df, table_name = 'CHEMBL_DATABASE', auto_create_table = True, overwrite = True)

### Calculating Molecular Descriptors: 

In [ ]:
def udf_compute_molecular_properties(smile_string_col: str) -> dict: 
    """Computes molecular properties of an input smile string, 
    returns dictionary with descriptor name and its value"""
    
    #Load the libraries:
    from rdkit import Chem
    from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
    
    #Descriptors to calculate smiles strings from: 
    descriptors = ['ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 
               'FractionCSP3', 'HallKierAlpha', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Ipc', 'Kappa1', 
               'Kappa2', 'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MaxEStateIndex', 
               'MaxPartialCharge', 'MinAbsEStateIndex', 'MinAbsPartialCharge', 'MinEStateIndex', 'MinPartialCharge',
               'MolLogP', 'MolMR', 'MolWt']
    
    #Compute molecular represntation
    mol = Chem.MolFromSmiles(smile_string_col)
    descriptor_calculator = MolecularDescriptorCalculator(descriptors)
    mol_property_vals = descriptor_calculator.CalcDescriptors(mol)
    return dict(zip(descriptors, mol_property_vals))

In [ ]:
udf_compute_molecular_properties = session.udf.register(func = udf_compute_molecular_properties, 
                                                        name = 'udf_compute_molecular_properties',
                                                        stage_location = '@TMP', 
                                                        replace = True, 
                                                        is_permanent = True, 
                                                        packages = ['cachetools==4.2.2', 'numpy==1.23.5', 'pandas==1.4.3', 'rdkit==2022.09.4'],
                                                        session = session
                                                       )

In [ ]:
session.table('CHEMBL_DATABASE')

In [ ]:
smiles_df.show()

In [ ]:
#Call UDF on data: 
smiles_df.with_column('MOLECULAR_PROPERTIES', 
                       udf_compute_molecular_properties(F.col('"Smiles"'))).limit(10).show()

The above query, when run against the full dataset, takes 46 minutes on an Extra-Small Warehouse on nearly 2 million records. To speed up this process, we will make use of vectorized UDFs. This emans that each call to the UDF receives a set/batch of rows compared to a Scalar UDF which gets one row input at a time. Combined with a larger sized warehouse, we can speed up the query execution time. 

In [ ]:
@cachetools.cached(cache={})
def get_molecular_descriptor_calculator():
    """Cache call to retreive molecular descriptors function"""
    from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
    
    descriptor_vals = ['ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 
               'FractionCSP3', 'HallKierAlpha', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Ipc', 'Kappa1', 
               'Kappa2', 'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MaxEStateIndex', 
               'MaxPartialCharge', 'MinAbsEStateIndex', 'MinAbsPartialCharge', 'MinEStateIndex', 'MinPartialCharge',
               'MolLogP', 'MolMR', 'MolWt']
    
    return MolecularDescriptorCalculator(descriptor_vals), descriptor_vals


def vudf_compute_molecular_properties(smile_string_df: T.PandasDataFrame[str]) -> T.PandasSeries[dict]: 
    """Vectorized implementation of the compute molecular properties, for more efficient processing"""
        
    #Load the libraries:
    from rdkit import Chem
    #from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
    
    smile_string_df.columns = ['SMILES']
    descriptor_calculator, descriptors = get_molecular_descriptor_calculator()
    
    def smiles_to_descriptors(smile):
        """Helper function to apply to batches of rows"""
        mol = Chem.MolFromSmiles(smile)
        mol_property_vals = descriptor_calculator.CalcDescriptors(mol)
        return dict(zip(descriptors, mol_property_vals))
    
    #smile_string_df['descriptors'] = smile_string_df.SMILES.apply(smi_to_descriptors)
    return smile_string_df.SMILES.apply(smiles_to_descriptors)

In [ ]:
#Register the vectorized udf:
vudf_compute_molecular_properties = session.udf.register(func = vudf_compute_molecular_properties, 
                                                        name = 'vudf_compute_molecular_properties',
                                                        stage_location = '@TMP', 
                                                        replace = True, 
                                                        is_permanent = True, 
                                                        packages = ['cachetools==4.2.2', 'numpy==1.23.5', 'pandas==1.4.3', 'rdkit==2022.09.4'],
                                                        session = session,
                                                        max_batch_size = 500
                                                       )

In [ ]:
#invoke the vectorized udf
smiles_df.with_column('MOLECULAR_PROPERTIES', 
                       vudf_compute_molecular_properties(F.col('"Smiles"'))).limit(10).show()

### Molecular Similarity with MACCS & Tanimoto Coefficient: 

The data used for this portion is sourced from: https://github.com/greglandrum/rdkit_blog/blob/master/data/chembl16_25K.pairs.txt.gz. 

In [ ]:
#Load the data: 
pairs_df = pd.read_table('/Users/hapatel/Downloads/chembl16_25K.pairs.txt', delimiter= ' ',
              names = ['pair1_id', 'smile_string1', 'pair2_id', 'smile_string2'])

pairs_df.head()

In [ ]:
session.write_pandas(pairs_df, table_name = "CHEMBL16_PAIRS", auto_create_table=True, overwrite=True)

In [ ]:
pairs_sdf = session.table('CHEMBL16_PAIRS')
pairs_sdf.show()

In [ ]:
def udf_compute_maccs_tanimoto_similarity(smile_string_col1: str, smile_string_col2: str) -> float:
    """Computes the tanimoto_similarity of two smile string compounds using the MACCS fingerprint"""
    
    #Load libraries
    from rdkit import Chem
    from rdkit.Chem import MACCSkeys
    from rdkit import DataStructs
    
    #Compute molecular representation
    mol1 = Chem.MolFromSmiles(smile_string_col1)
    mol2 = Chem.MolFromSmiles(smile_string_col2)
    
    #Generate the maccs keys for smile string
    maccs1 = MACCSkeys.GenMACCSKeys(mol1)
    maccs2 = MACCSkeys.GenMACCSKeys(mol2)
    
    return DataStructs.TanimotoSimilarity(maccs1, maccs2)

In [ ]:
udf_maccs_tanimoto_similarity = session.udf.register(func = udf_compute_maccs_tanimoto_similarity, 
                                                        name = 'udf_compute_maccs_tanimoto_similarity',
                                                        stage_location = '@TMP', 
                                                        replace = True, 
                                                        is_permanent = True, 
                                                        packages = ['cachetools==4.2.2', 'numpy==1.23.5', 'pandas==1.4.3', 'rdkit==2022.09.4'],
                                                        session = session
                                                       )

In [ ]:
pairs_sdf.with_column('MACCS_Tanimoto_similarity', udf_maccs_tanimoto_similarity(F.col('"smile_string1"'), 
                                                                                    F.col('"smile_string2"')))\
             .limit(100).show()

### Substructure Search: 

In [ ]:
def udf_has_substructure_match(smile_string_col: str, pattern: str) -> bool:
    """Given a molecule represented as a smile string and a user defined pattern, 
    will return True if pattern is within the input smile string"""
    
    #Load libraries
    from rdkit import Chem
    
    #Compute molecular representation
    mol = Chem.MolFromSmiles(smile_string_col)
    patt = Chem.MolFromSmarts(pattern)
    
    return mol.HasSubstructMatch(patt)

In [ ]:
udf_has_substructure_match = session.udf.register(func = udf_has_substructure_match, 
                                                        name = 'udf_has_substructure_match',
                                                        stage_location = '@TMP', 
                                                        replace = True, 
                                                        is_permanent = True, 
                                                        packages = ['cachetools==4.2.2', 'numpy==1.23.5', 'pandas==1.4.3', 'rdkit==2022.09.4'],
                                                        session = session
                                                       )

In [ ]:
smiles_df.with_column('has_substructure', 
                       udf_has_substructure_match(F.col('"Smiles"'), F.lit('ccO'))).limit(10).show()